# Quick Start: English to Spanish Steering Vector

This notebook demonstrates how to use the `steering-vectors` library to optimize a steering vector that causes a language model to generate Spanish text instead of English.

This is a direct adaptation of the original `llm-steering-opt/quickstart.ipynb` using the new modular API.

## Setup

In [53]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from dotenv import load_dotenv
import os

# Import from the new steering-vectors library
from steering_vectors import (
    SteeringOptimizer,
    VectorSteering,
    ClampSteering,
    HuggingFaceBackend,
    TrainingDatapoint,
    OptimizationConfig,
)

load_dotenv()

True

In [54]:
hf_token = os.getenv("HF_TOKEN")

model_name = "google/gemma-2-2b"

tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
model = AutoModelForCausalLM.from_pretrained(model_name, dtype=torch.bfloat16, token=hf_token)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [56]:
# Move to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
print(f"Using device: {device}")
backend = HuggingFaceBackend(model, tokenizer, device)

Using device: cuda


## Task Definition: English to Spanish Switching

We'll optimize a steering vector that causes the model to generate Spanish instead of English, given an English prompt.

In [57]:
# Our training prompt - a recipe introduction
prompt = """Some of my fondest childhood memories are from my summer vacations back when I was little. Every now and then, after a long day of playing outside, I would come back home to be greeted with the delicious smell of my grandma's hazelnut cake wafting out of the kitchen. In this recipe, I'll teach you how to make that very cake, and create your own summer memories.

"""

In [58]:
# Generate an unsteered completion to see what the model normally produces
generated = backend.generate(
    prompt,
    max_new_tokens=15,
    do_sample=False
)
print("Unsteered completion:")
print(generated_str.replace(prompt, ""))

Unsteered completion:
<h2>Ingredients</h2>

* 1 cup of butter
* 1 cup


In [59]:
# Define our completions: suppress English, promote Spanish
en_completion = """<h2>Ingredients</h2>

* 1 cup of all-purpose flour"""

es_completion = """<h2>Ingredientes</h2>

* 1 taza de harina común"""

## Optimizing the Steering Vector

Now we use the `steering-vectors` library's API:
1. Create a `HuggingFaceBackend` to handle model operations
2. Create a `VectorSteering` mode for additive steering
3. Configure with `OptimizationConfig`
4. Run optimization with `SteeringOptimizer`

In [60]:
# Create our training datapoint
datapoint = TrainingDatapoint(
    prompt=prompt,
    src_completions=[en_completion],  # Suppress English
    dst_completions=[es_completion],  # Promote Spanish
)

In [61]:
# Set up the components
backend = HuggingFaceBackend(model, tokenizer)
steering = VectorSteering()

# Configure optimization (matching original: lr=0.1, max_iters=20)
config = OptimizationConfig(
    lr=0.1,
    max_iters=20,
)

# Create optimizer
optimizer = SteeringOptimizer(backend, steering, config)

In [62]:
# Run optimization at layer 10
layer = 10
result = optimizer.optimize([datapoint], layer=layer)

print(f"Optimization complete!")
print(f"  Iterations: {result.iterations}")
print(f"  Final loss: {result.final_loss:.4f}")
print(f"  Vector norm: {steering.get_vector().norm().item():.2f}")

Optimization complete!
  Iterations: 20
  Final loss: 4.0312
  Vector norm: 46.75


## Testing the Steering Vector

Let's see if our optimized vector causes the model to generate Spanish!

In [63]:
# Create the steering hook
hook = steering.create_hook()

# Generate with steering
steered_str = backend.generate_with_steering(
    prompt,
    steering_mode=steering,
    layers=layer,
    max_new_tokens=30,
    do_sample=False
)

print("Steered completion:")
print(steered_str)

Steered completion:
Some of my fondest childhood memories are from my summer vacations back when I was little. Every now and then, after a long day of playing outside, I would come back home to be greeted with the delicious smell of my grandma's hazelnut cake wafting out of the kitchen. In this recipe, I'll teach you how to make that very cake, and create your own summer memories.

<h2>Ingredientes</h2>

* 1 taza de harina común
* 1 taza de azúcar
* 1 taza de leche
* 1 cucharadita


### Generalization Test

Does the vector generalize to other prompts?

In [64]:
test_prompt = "My favorite programming"
max_new_tokens = 35

print("--- Unsteered generation ---")
unsteered_text = backend.generate(
    test_prompt,
    max_new_tokens=max_new_tokens,
    do_sample=False
)
print(unsteered_text)
print()

print("--- Steered generation ---")
steered_text = backend.generate_with_steering(
    test_prompt,
    steering_mode=steering,
    layers=layer,
    max_new_tokens=max_new_tokens,
    do_sample=False
)
print(steered_text)

--- Unsteered generation ---
My favorite programming language is Python. I love the simplicity of the language and the fact that it is easy to learn. I also love the fact that it is a general-purpose language,

--- Steered generation ---
My favorite programming lenguaje es el C, pero me gustaría aprender el C++ y el Java.

I like the C, pero me gustaría aprender el C++ y el Java.

I


### Reverse Steering

If we negate the vector, we can switch from Spanish to English!

In [65]:
spanish_prompt = "Unos de mis lenguajes de programación favoritos incluyen"
max_new_tokens = 30

print("--- Unsteered generation (Spanish prompt) ---")
spanish_unsteered = backend.generate(spanish_prompt, max_new_tokens=max_new_tokens, do_sample=False)
print(spanish_unsteered)
print()

print("--- Reverse-steered generation (should switch to English) ---")

# Use strength=-1 to negate the vector
reverse_steered = backend.generate_with_steering(
    spanish_prompt,
    steering_mode=steering,
    layers=layer,
    strength=-1.0,
    max_new_tokens=max_new_tokens,
    do_sample=False
)
print(reverse_steered)

--- Unsteered generation (Spanish prompt) ---
Unos de mis lenguajes de programación favoritos incluyen Python, Java, C++, C#, y JavaScript.

Python es un lenguaje de programación muy popular que se utiliza para crear aplicaciones web, juegos,

--- Reverse-steered generation (should switch to English) ---
Unos de mis lenguajes de programación favoritos incluyen Python, C#, and C++.

I’ve been using Python for a long time, and I’ve been using C# for a long


In [67]:
test_prompt_2 = '"How dare you cheat on me with him!" Jim roared.'

print("--- Unsteered generation ---")
generated = backend.generate(
    test_prompt_2,
    max_new_tokens=max_new_tokens,
    do_sample=False
)
print(generated)
print()

print("--- Steered generation (steering caused incoherent text)---")
generated = backend.generate_with_steering(
    test_prompt_2,
    steering_mode=steering,
    layers=layer,
    max_new_tokens=max_new_tokens,
    do_sample=False
)

print(generated)

--- Unsteered generation ---
"How dare you cheat on me with him!" Jim roared.

"I'm sorry, Jim. I didn't mean to. I just wanted to be with him."

"You're a

--- Steered generation (steering caused incoherent text)---
"How dare you cheat on me with him!" Jim roared.

"No, no, no, no, no, no, no, no, no, no, no, no, no, no,


## Norm-Constrained Steering

We can limit the vector's norm to prevent overly strong steering effects.

In [68]:
# Create a new steering mode and optimizer with norm constraint
steering_constrained = VectorSteering()
config_constrained = OptimizationConfig(
    lr=0.1,
    max_iters=20,
    max_norm=20.0,  # Limit vector norm to 20
)

optimizer_constrained = SteeringOptimizer(backend, steering_constrained, config_constrained)
result_constrained = optimizer_constrained.optimize([datapoint], layer=layer)

print(f"Norm-constrained optimization:")
print(f"  Final loss: {result_constrained.final_loss:.4f}")
print(f"  Vector norm: {steering_constrained.get_vector().norm().item():.2f}")

Norm-constrained optimization:
  Final loss: 7.0625
  Vector norm: 20.00


In [69]:
# Test the norm-constrained vector
test_prompt = "My favorite programming languages are"
hook_constrained = steering_constrained.create_hook()

print("--- Steered with norm-constrained vector ---")

generated = backend.generate(
    test_prompt,
    max_new_tokens=max_new_tokens,
    hooks=[(layer, hook_constrained)],
    do_sample=False,
)

print(generated)

--- Steered with norm-constrained vector ---
My favorite programming languages are:

* <strong>Python</strong>: es un lenguaje de programación de alto nivel, orientado a objetos, multiplataforma, libre y de código


In [71]:
test_prompt_2 = '"How dare you cheat on me with him!" Jim roared.'

print("--- Unsteered generation ---")
generated = backend.generate(
    test_prompt_2,
    max_new_tokens=max_new_tokens,
    do_sample=False
)
print(generated)
print()

print("--- Steered generation (still doesn't work) ---")
generated = backend.generate(
    test_prompt_2,
    max_new_tokens=max_new_tokens,
    hooks=[(layer, hook_constrained)],
    do_sample=False,
)

print(generated)

--- Unsteered generation ---
"How dare you cheat on me with him!" Jim roared.

"I'm sorry, Jim. I didn't mean to. I just wanted to be with him."

"You're a

--- Steered generation (still doesn't work) ---
"How dare you cheat on me with him!" Jim roared.

"Jim, I'm sorry, I didn't mean to, I just..."

"You just what? You just cheated on me


## Clamp Steering

In [83]:
clamp_steering = ClampSteering()

clamp_optimizer = SteeringOptimizer(backend, clamp_steering, config)

clamped_result = clamp_optimizer.optimize([datapoint], layer=layer)

print(f"Optimization complete!")
print(f"  Iterations: {clamped_result.iterations}")
print(f"  Final loss: {clamped_result.final_loss:.4f}")
print(f"  Vector norm: {clamp_steering.get_vector().norm().item():.2f}")

Optimization complete!
  Iterations: 20
  Final loss: 1.8398
  Vector norm: 32.25


In [86]:
print("--- Steered generation ---")

generated = backend.generate_with_steering(
    test_prompt,
    steering_mode=clamp_steering,
    layers=layer,
    max_new_tokens=max_new_tokens,
    do_sample=True,
)

print(generated)
print()

print("--- Steered generation ---")
generated = backend.generate_with_steering(
    test_prompt_2,
    steering_mode=clamp_steering,
    layers=layer,
    max_new_tokens=max_new_tokens,
    do_sample=True, # with do_sample=False steering didn't work
)

print(generated)


--- Steered generation ---
My favorite programming languages are:

* <em>Java</em> - Java tiene una buena comunidad de desarrolladores, tiene buenas herramientas como Eclipse, etc. Es uno de los mejores

--- Steered generation ---
"How dare you cheat on me with him!" Jim roared.

"Qué, tú crees que voy a quedar aquí... Me voy!"

"Tú le metiste en esto, tú tenías que cuidarlo
